# this Notebook shows how to measure the road route distance betweeen two places if cordinates are given

In [137]:
import pandas as pd # to load cities dataset
import requests
import json
from geopy import distance # to calculate distance on the surface

# need to test
!pip install osrm
import osrm
result = osrm.simple_route([21.0566163803209,42.004088575972], [20.9574645547597, 41.5286973392856],
                      output='route', overview="full", geometry='wkt')

In [140]:
data=pd.read_excel(r"C:\Users\rmallik\Downloads\New folder\data\ap.xlsx")
data

8.481053325259792

# displacement on surface

In [141]:
#
d = distance.distance((13.009241,77.551631), (13.056618,77.613100))
d.km

8.481053325259792

# Route Distance 

# reference http://project-osrm.org/docs/v5.5.1/api/#route-service


In [142]:
#https://stackoverflow.com/questions/48221046/osrm-giving-wrong-response-for-distance-between-2-points
# cordinates should be long,latitude order

url=f"""https://router.project-osrm.org/route/v1/driving/{data.loc[0, "School Location_langitude"]},{data.loc[0, "School Location_latitude"]};{data.loc[10, "School Location_langitude"]},{data.loc[10, "School Location_latitude"]}?overview=false"""
url='http://router.project-osrm.org/route/v1/driving/77.551631,13.009241;77.6131,13.056618?overview=false'
url

r = requests.get(url)
r.content

payload = {"steps":"true","geometries":"geojson"}

response = requests.get(url,params=payload)

routes = json.loads(response.content)
route_1 = routes.get("routes")[0]
route_1["distance"]


## route distance to sum of all the distance travelled by car

In [132]:
Result_list=[]
prev_sl=0
for rt in data["Route name"].unique():
    data_rt=data[data["Route name"]==rt]
    dist_rt=0
    
    for i in range(len(data_rt)-1) :            
            #print(data_rt.loc[i, "School Location_latitude"],data_rt.loc[i, "School Location_langitude"],data_rt.loc[i+1, "School Location_latitude"],data_rt.loc[i+1, "School Location_langitude"])
            url=f"""https://router.project-osrm.org/route/v1/driving/{data_rt.loc[i+prev_sl, "School Location_latitude"]},{data_rt.loc[i+prev_sl, "School Location_langitude"]};{data_rt.loc[i+prev_sl+1, "School Location_latitude"]},{data_rt.loc[i+prev_sl+1, "School Location_langitude"]}?overview=false"""
            response=requests.get(url)
            routes = json.loads(response.content)
            route_1 = routes.get("routes")[0]
            dist_v=route_1["distance"]
            dist_rt+=float(dist_v)
            #print(url,dist_v)
    prev_sl=prev_sl+len(data_rt)
    Result_list.append([rt,dist_rt])


In [134]:
df=pd.DataFrame(Result_list,columns=["Route_name","Distance"])
df.to_excel("output.xlsx")

In [135]:
df

,Route_name,Distance
0,Kothanur,152474.0
1,LR Puram,41504.0
